In [1]:
import seaborn as sns
import pandas as pd
from tensorflow.python.keras import Input
from evaluator import calculate_metrics
from kmeans import kmeans
from yolov3_tf2 import dataset
import tensorflow as tf
from yolov3_tf2.models import YoloV3, DarknetConv, DarknetBlock
import matplotlib.pyplot as plt
import os
from yolov3_tf2.utils import draw_outputs
import numpy as np
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from PIL import Image
sns.set_style("whitegrid")
sns.despine(offset=10, trim=True)

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

ModuleNotFoundError: No module named 'evaluator'

In [ ]:
pix_coord = '/data2/seals/pixel_coord.xlsx'
og_records = pd.read_excel(pix_coord, sheet_name="PixelCoordinates")[["tiff_file", "x_pixel", "y_pixel"]].dropna()
file_props = pd.read_excel(pix_coord, sheet_name="FileOverview").dropna()
og_records = og_records.merge(
    file_props[["tiff_file", "image_width", "image_height"]], how="inner"
)
og_records.head()

In [ ]:
normalised_og = og_records.copy()
normalised_og['x_pixel'] = normalised_og['x_pixel'] / normalised_og['image_width']
normalised_og['y_pixel'] = normalised_og['image_height'] - normalised_og['y_pixel']
normalised_og['y_pixel'] = normalised_og['y_pixel'] / normalised_og['image_height']
normalised_og.head()

In [ ]:
sns.distplot(normalised_og["x_pixel"])

In [ ]:
def extract_intervals(size: tuple, split_size: tuple):
    """
    A function to generate pixel intervals
    :param size: the tuple representing (length, height)
    :param split_size: the split size
    :return:
    """
    og_width, og_height = size[0], size[1]
    if og_width < split_size[0] or og_height < split_size[1]:
        raise ValueError("The split size is larger than the image")
    result_widths = [
        (i, i + split_size[0])
        for i in range(0, og_width, split_size[0])
        if (split_size[0] + i) <= og_width
    ]
    result_heights = [
        (i, i + split_size[1])
        for i in range(0, og_height, split_size[1])
        if (split_size[1] + i) <= og_height
    ]
    return result_widths, result_heights

In [ ]:
Image.MAX_IMAGE_PIXELS = 99999999999999999999
with Image.open('') as img:
    intervals = extract_intervals(img.size, (416, 416))
    intervals = [[x[0], y[0], x[1], y[1]] for x, y in zip(intervals[0], intervals[1])]
    img_arr = np.array(img)
    draw_outputs

In [ ]:
sns.jointplot(x="x_pixel", y="y_pixel", data=normalised_og.round(1))

In [ ]:
all_records = pd.read_csv('/data2/seals/tfrecords/416/416_test_all_records.csv')
all_records.head()

In [ ]:
sns.distplot(all_records['x_pixel'])

In [ ]:
normalised = all_records.copy()
normalised['x_pixel'] = normalised['x_pixel'].apply(lambda x: x / 416)
normalised['y_pixel'] = normalised['y_pixel'].apply(lambda y: y / 416)
sns.jointplot(x="x_pixel", y="y_pixel", data=normalised.round(1))

In [ ]:
anchors_og = kmeans(normalised_og[["x_pixel", "y_pixel"]].to_numpy(), 9)
sns.scatterplot(x=anchors_og[:, 0], y=anchors_og[:, 1])

In [ ]:
anchors = kmeans(normalised[["x_pixel", "y_pixel"]].to_numpy(), 9)

In [ ]:
full_dim = np.vectorize(lambda x: x * 416)
full_anchors = full_dim(anchors)
sns.scatterplot(x=anchors[:, 0], y=anchors[:, 1])

## Developing the 1024 model
### backbone

In [ ]:
def Darknet(name=None):
    x = inputs = Input([None, None, 3])
    x = DarknetConv(x, 32, 3)
    x = DarknetBlock(x, 64, 1)
    x = DarknetBlock(x, 128, 2)  # skip connection
    x = x_36 = DarknetBlock(x, 256, 8)  # skip connection
    x = x_61 = DarknetBlock(x, 512, 8)
    x = DarknetBlock(x, 1024, 4)
    return tf.keras.Model(inputs, (x_36, x_61, x), name=name)

backbone = ResNet152V2(input_shape=(1024, 1024, 3), include_top=False)
backbone.summary()

In [ ]:
backbone.get_layer('conv5_block3_out').output

# Evaluation

Evaluating the model accuracy.

## The Jaccard Index

In object detection it is unlikely that the predicted bounding boxes align perfectly with the ground truth data.
The Jaccard index or the Intersection over Union IoU score gives an estimate of how closely the boxes match our ground truth.

$IoU = \frac{area(B_p \cap B_{gt})}{area(B_p \cup B_{gt})}$

### Loading the train dataset

In [ ]:
train_data_eval = dataset.load_tfrecord_dataset('/data2/seals/tfrecords/416/train.tfrecord', './data/data_files/seal.names', 416)
train_data_eval = train_data_eval.map(lambda x, y: (tf.expand_dims(x, 0), y,))
train_data_eval = train_data_eval.map(lambda x, y: (dataset.transform_images(x, 416), y,))

In [ ]:
data_eval = dataset.load_tfrecord_dataset('/data2/seals/tfrecords/416/test.tfrecord',
                                          './data/data_files/seal.names', 416)
data_eval = data_eval.map(lambda x, y: (tf.expand_dims(x, 0), y,))
data_eval = data_eval.map(lambda x, y: (dataset.transform_images(x, 416), y,))

In [ ]:
model = YoloV3(classes=80)
model.load_weights('/home/md273/model_zoo/416_eager/checkpoints/yolov3_train_7.tf').expect_partial()

In [ ]:
metrics_train = calculate_metrics(train_data_eval, model)
metrics_test = calculate_metrics(data_eval, model)

In [ ]:
sns.distplot(metrics_train['ious'], label='train')
sns.distplot(metrics_test['ious'], label='test')
plt.legend()

In [ ]:
from PIL import Image
